In [4]:
!pip install -r /jupyter-tutorial/AAIPL_134_199_198_15/default_requirements.txt

  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl (821.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+rocm5.7
    Uninstalling torch-2.3.1+rocm5.7:━━━━━━━━━━━ 0/2 [torch]
      Successfully uninstalled torch-2.3.1+rocm5.70/2 [torch]
  Attempting uninstall: torchvision━━━━━━━━━━━━━ 0/2 [torch]
    Found existing installation: torchvision 0.18.1+rocm5.70m [torch]
    Uninstalling torchvision-0.18.1+rocm5.7: 0/2 [torch]
      Successfully uninstalled torchvision-0.18.1+rocm5.7 [torch]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchvision] [torchvision]
ERROR: pip's dependency resolver does not currently take into account all the packa

In [1]:
!rocm-smi



============================================ ROCm System Management Interface ============================================
====================================================== Concise Info ======================================================
Device  Node  IDs              Temp        Power     Partitions          SCLK    MCLK    Fan  Perf  PwrCap  VRAM%  GPU%  
              (DID,     GUID)  (Junction)  (Socket)  (Mem, Compute, ID)                                                  
0       1     0x74b5,   21947  38.0°C      154.0W    NPS1, SPX, 0        150Mhz  900Mhz  0%   auto  750.0W  0%     0%    
================================================== End of ROCm SMI Log ===================================================


In [2]:
import torch
print("Is a ROCm-GPU detected? ", torch.cuda.is_available())
print("How many ROCm-GPUs are detected? ", torch.cuda.device_count())

Is a ROCm-GPU detected?  True
How many ROCm-GPUs are detected?  1


In [7]:
!pwd

/jupyter-tutorial/AAIPL_134_199_198_15/tutorial


In [9]:
# Install `bitsandbytes` for ROCm 6.0+.
# Use -DBNB_ROCM_ARCH to target a specific GPU architecture.
!git clone --recurse https://github.com/ROCm/bitsandbytes.git
!cd jupyter-tutorial/AAIPL_134_199_198_15/tutorial/bitsandbytes
!git checkout rocm_enabled_multi_backend
!pip install -r requirements-dev.txt
!cmake -DBNB_ROCM_ARCH="gfx942" -DCOMPUTE_BACKEND=hip -S .
!python setup.py install

# To leverage the SFTTrainer in TRL for model fine-tuning.
!pip install trl

# To leverage PEFT for efficiently adapting pre-trained language models .
!pip install peft

# Install the other dependencies.
!pip install transformers datasets huggingface-hub scipy

fatal: destination path 'bitsandbytes' already exists and is not an empty directory.
/bin/bash: line 1: cd: jupyter-tutorial/AAIPL_134_199_198_15/tutorial/bitsandbytes: No such file or directory
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements-dev.txt'
CMake Error: The source directory "/jupyter-tutorial/AAIPL_134_199_198_15/tutorial" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
python: can't open file '/jupyter-tutorial/AAIPL_134_199_198_15/tutorial/setup.py': [Errno 2] No such file or directory


In [10]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer

RuntimeError: Failed to import transformers.models.bloom.modeling_bloom because of the following error (look up to see its traceback):
/opt/rocm/lib/libamdhip64.so.6: undefined symbol: hsa_amd_vmem_export_shareable_handle, version ROCR_1

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/jupyter-tutorial/hf_models/Qwen3-4B", padding_side='left')


In [16]:
# Dataset for fine-tuning.
training_dataset_name = "garage-bAInd/Open-Platypus"
training_dataset = load_dataset(training_dataset_name, split = "train")

# Check the data.
print(training_dataset)

# Dataset 11 is a QA sample in English.
print(training_dataset[11])

Dataset({
    features: ['input', 'output', 'instruction', 'data_source'],
    num_rows: 24926
})
{'input': '', 'output': "If a sequence contains no more than one 0, there are $7\\cdot 6\\cdot\n5\\cdot 4\\cdot 3 = 2520$ sequences formed from the characters A, I, M, E, 2, 0, and 7. If a sequence contains two 0's, the 0's can be placed in $\\binom{5}{2} = 10$ ways, the remaining characters can be chosen in $\\binom{6}{3} = 20$ ways, and those remaining characters can be arranged in $3! = 6$ ways, for a total of $10\\cdot 20\\cdot 6\n= 1200$ sequences. Thus $N = 2520 + 1200 = 3720$, and $\\frac{N}{10}=\n372$.", 'instruction': 'A mathematical organization is producing a set of commemorative license plates. Each plate contains a sequence of five characters chosen from the four letters in AIME and the four digits in 2007. No character may appear in a sequence more times than it appears among the four letters in AIME or the four digits in 2007. A set of plates in which each possible sequence 

In [17]:
# Training parameters for SFTTrainer.
from transformers import TrainingArguments
training_arguments = TrainingArguments(
    output_dir = "ckpt",
         num_train_epochs = 1,
         per_device_train_batch_size = 4,
         gradient_accumulation_steps = 1,
         optim = "paged_adamw_32bit",
         save_steps = 50,
         logging_steps = 50,
         learning_rate = 4e-5,
         weight_decay = 0.001,
         fp16=False,
         bf16=False,
         max_grad_norm = 0.3,
         max_steps = -1,
         warmup_ratio = 0.03,
         group_by_length = True,
         lr_scheduler_type = "constant",
         report_to = "tensorboard",
    no_cuda=True
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [18]:
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("/jupyter-tutorial/hf_models/Qwen3-4B",torch_dtype=torch.bfloat16,device_map="auto")

from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # ✅ manually specify
)

from peft import get_peft_model
peft_model = get_peft_model(base_model, peft_config)
peft_model.print_trainable_parameters()

RuntimeError: Failed to import transformers.models.qwen3.modeling_qwen3 because of the following error (look up to see its traceback):
/opt/rocm/lib/libamdhip64.so.6: undefined symbol: hsa_amd_vmem_export_shareable_handle, version ROCR_1

In [66]:
import torch

print("HIP (AMD ROCm) version:", torch.version.hip)      # e.g., "6.3.4..."
print("Is CUDA available:", torch.cuda.is_available())   # True if ROCm is working
print("GPU name:", torch.cuda.get_device_name(0))        # e.g., "MI300X"


HIP (AMD ROCm) version: 6.3.42133-1b9c17779
Is CUDA available: True


RuntimeError: CUDA driver error: 100

In [2]:
print("GPU Name:", torch.cuda.get_device_name(0))

GPU Name: AMD Instinct MI300X VF


In [ ]:
# Initialize an SFT trainer.
sft_trainer = SFTTrainer(
        model = base_model,
        train_dataset = training_dataset,
        peft_config = peft_config,
        dataset_text_field = "text",
        tokenizer = tokenizer,
        args = training_arguments
)

# Run the trainer.
sft_trainer.train()

In [56]:
# PEFT adapter name.
adapter_name = "Qwen3-4B-enhanced-adapter"

# Save PEFT adapter.
sft_trainer.model.save_pretrained(adapter_name)

NameError: name 'sft_trainer' is not defined

In [ ]:
# Access adapter directory.
cd Qwen3-4B-enhanced-adapter

# List all adapter files.
README.md  adapter_config.json  adapter_model.safetensors

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Set the path of the model or the name on Hugging face hub
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

# Set the path of the adapter
adapter_name = "Llama-2-7b-enhanced-adpater"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Adapt the base model with the adapter
new_model = PeftModel.from_pretrained(base_model, adapter_name)

# Then, run generation as the same with a normal model outlined in 2.1

In [ ]:
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Adapt the base model with the adapter
new_model = PeftModel.from_pretrained(base_model, adapter_name)

# Merge adapter
model = model.merge_and_unload()

# Save the merged model into local
model.save_pretrained("merged_adpaters")